# [Base Model Selection](https://learn.microsoft.com/azure/ai-foundry/concepts/evaluation-approach-gen-ai#base-model-selection)

The first stage of the AI lifecycle involves selecting an appropriate base model. Generative AI models vary widely in terms of capabilities, strengths, and limitations, so it's essential to identify which model best suits your specific use case. During base model evaluation, you "shop around" to compare different models by testing their outputs against a set of criteria relevant to your application.

You have three options to evaluate models:

1. [Use Azure AI Foundry Benchmarks](https://learn.microsoft.com/en-us/azure/ai-foundry/concepts/model-benchmarks) to compare models on their intrinsic capabilities.
1. [Use Manual Evaluations in the Portal](https://learn.microsoft.com/en-us/azure/ai-foundry/how-to/evaluate-prompts-playground) to run prompts on models and rate them.
1. [Evaluate Multiple Models using the SDK](https://learn.microsoft.com/en-us/azure/ai-foundry/how-to/evaluate-prompts-playground) code-first

**In this notebook, we explore a simplified version of option 3**

---


## Objective

You are beginning your AI application development journey - and you have two (or more) model options available to you. How do you pick the right one for your needs? In this tutorial we look at how you can evaluate _the same set of prompts_ against multiple model endpoints deployed in your Azure AI project.

This guide uses Python Class as an application target which is passed to Evaluate API provided by PromptFlow SDK to evaluate results generated by LLM models against provided prompts. 

This tutorial uses the following Azure AI services:

- [azure-ai-evaluation](https://learn.microsoft.com/en-us/azure/ai-studio/how-to/develop/evaluate-sdk)

## Time

You should expect to spend 30 minutes running this sample. 

## About this example

This example demonstrates evaluating model endpoints responses against provided prompts using azure-ai-evaluation

## Before you begin

### Installation

Install the following packages required to execute this notebook. 

### Validate Required Environment Variables are set

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
import os

assert os.environ.get("AZURE_OPENAI_ENDPOINT") is not None, "Please set the AZURE_OPENAI_ENDPOINT environment variable."
assert os.environ.get("AZURE_OPENAI_API_VERSION") is not None, "Please set the AZURE_OPENAI_API_VERSION environment variable."
assert os.environ.get("AZURE_OPENAI_API_KEY") is not None, "Please set the AZURE_OPENAI_API_KEY environment variable."
assert os.environ.get("AZURE_AI_CONNECTION_STRING") is not None, "Please set the AZURE_AI_CONNECTION_STRING environment variable."

assert os.environ.get("AZURE_OPENAI_GPT4O_MINI_DEPLOYMENT_NAME") is not None, "Please set the AZURE_OPENAI_GPT4O_MINI_DEPLOYMENT_NAME environment variable."
assert os.environ.get("AZURE_OPENAI_GPT35_DEPLOYMENT_NAME") is not None, "Please set the AZURE_OPENAI_GPT35_MINI_DEPLOYMENT_NAME environment variable."
assert os.environ.get("LLM_JUDGE_DEPLOYMENT_NAME") is not None, "Please set the LLM_JUDGE_DEPLOYMENT_NAME environment variable."

## Model Evaluation

We will use Evaluate API provided by Azure AI Evaluation SDK. In the notebook, we will use different models and evaluate them. Azure AI Foundry will be used to visualize and compare results


#### Getting Azure AI Foundry Project details

In [3]:
import os

# Project Connection String
connection_string = os.environ.get("AZURE_AI_CONNECTION_STRING")

# Extract details
region_id, subscription_id, resource_group_name, project_name = connection_string.split(";")

# Populate it
azure_ai_project = {
    "subscription_id": subscription_id,
    "resource_group_name": resource_group_name,
    "project_name": project_name,
}
print(azure_ai_project)

{'subscription_id': 'fac34303-435d-4486-8c3f-7094d82a0b60', 'resource_group_name': 'rg-anksingai', 'project_name': 'anksing-eval-agents'}


## Data

Following code reads Json file "data.jsonl" which queries that will passed to each model for evaluation. 

In [4]:
import pandas as pd

df = pd.read_json("data.jsonl", lines=True)
print(df.head())

                                               query  \
0                     When was United Stated found ?   
1                     What is the capital of France?   
2                 Which tent is the most waterproof?   
3         Which camping table holds the most weight?   
4  What is the weight of the Adventure Dining Table?   

                                        ground_truth  
0                                               1776  
1                                              Paris  
2  The Alpine Explorer Tent has the highest rainf...  
3  The Adventure Dining Table has a higher weight...  
4           The Adventure Dining Table weighs 15 lbs  


## Configuration
To use Relevance and Cohenrence Evaluator, we will Azure Open AI model details as a Judge that can be passed as model config.

In [5]:
from azure.ai.evaluation import AzureOpenAIModelConfiguration

# Note: We are evaluating 2 models above - and we need a "LLM Judge" to evaluate them
#       Here we specify the judge model
model_config = AzureOpenAIModelConfiguration(
    azure_endpoint=os.environ.get("AZURE_OPENAI_ENDPOINT"),
    azure_deployment=os.environ.get("LLM_JUDGE_DEPLOYMENT_NAME"),
    api_key=os.environ.get("AZURE_OPENAI_API_KEY"),
    api_version=os.environ.get("AZURE_OPENAI_API_VERSION"),
)
print(model_config)

[INFO] Could not import AIAgentConverter. Please install the dependency with `pip install azure-ai-projects`.
{'azure_endpoint': 'https://ai-anksingevalhub437059299020.openai.azure.com', 'azure_deployment': 'gpt-4', 'api_key': '4DOT2GVBei4eOY4KnpgxkQoNw9BuXmV0jfwzRNVLaSJbZAQYMpeeJQQJ99BBACHYHv6XJ3w3AAAAACOGxfAw', 'api_version': '2024-12-01-preview'}


## Run the evaluation

The Following code runs Evaluate API and Relevance, Coherence (LLM as Judge), Bleu, Rogue (NLP) and Violence (Content Safety) Evaluator to evaluate results from different models.

The following are the few parameters required by Evaluate API. 

+   Data file (Prompts): It represents data file 'data.jsonl' in JSON format. Each line contains question, context and ground truth for evaluators.     

+   Application Target: It is name of python class which calls the model.  

+   Model Name: It is an identifier of model so that custom code in the App Target class can identify the model type and call respective LLM model using endpoint URL and auth key.  

+   Evaluators: List of evaluators is provided, to evaluate given prompts (questions) as input and output (answers) from LLM models. 

### Initialize the evaluators  

In [6]:
import pathlib

from azure.ai.evaluation import evaluate
from azure.ai.evaluation import (
    RelevanceEvaluator, CoherenceEvaluator, BleuScoreEvaluator, RougeScoreEvaluator, RougeType, ViolenceEvaluator,
)
from azure.identity import DefaultAzureCredential

# LLM as judge evaluator
relevance_evaluator = RelevanceEvaluator(model_config)
coherence_evaluator = CoherenceEvaluator(model_config)

# NLP evaluators
blue_score_evaluator = BleuScoreEvaluator()
rouge_score_evaluator = RougeScoreEvaluator(rouge_type=RougeType.ROUGE_1)

# Violence evaluator
violence_evaluator = ViolenceEvaluator(azure_ai_project=azure_ai_project, credential=DefaultAzureCredential())

# Define the models to evaluate


path = str(pathlib.Path(pathlib.Path.cwd())) + "/data.jsonl"
print(path)

Class ViolenceEvaluator: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.


c:\Users\anksing\Repos\singankit\BUILD25-LAB334\notebooks\01-select-model/data.jsonl


### Function that uses `evaluate` API from `azure-ai-evaluation` package to Evaluate a Base Model

In [7]:
from model_endpoints import BaseModel

def evaluate_model(model):
    results = evaluate(
        evaluation_name=f"Base Model Evaluation {model}",
        data=path,
        target=BaseModel(model),
        evaluators={
            "relevance": relevance_evaluator,
            "coherence": coherence_evaluator,
            "blue_score": blue_score_evaluator,
            "rouge_score": rouge_score_evaluator,
            "violence_score": violence_evaluator,
        },
        azure_ai_project=azure_ai_project,
    )
    return results
    

### Evaluate GPT 35 Turbo

In [8]:
gpt_35_turbo_results = evaluate_model("gpt-35-turbo")
pd.DataFrame(gpt_35_turbo_results["rows"])

[2025-05-09 15:28:12 -0700][promptflow._sdk._orchestrator.run_submitter][INFO] - Submitting run azure_ai_evaluation_evaluators_TARGET_20250509_152807_846057, log path: C:\Users\anksing\.promptflow\.runs\azure_ai_evaluation_evaluators_TARGET_20250509_152807_846057\logs.txt


2025-05-09 15:28:12 -0700   24532 execution.bulk     INFO     Current thread is not main thread, skip signal handler registration in BatchEngine.
2025-05-09 15:28:12 -0700   24532 execution          WARNING  Starting run without column mapping may lead to unexpected results. Please consult the following documentation for more information: https://aka.ms/pf/column-mapping
2025-05-09 15:28:12 -0700   24532 execution.bulk     INFO     Current system's available memory is 44050.62890625MB, memory consumption of current process is 231.7109375MB, estimated available worker count is 44050.62890625/231.7109375 = 190
2025-05-09 15:28:12 -0700   24532 execution.bulk     INFO     Set process count to 4 by taking the minimum value among the factors of {'default_worker_count': 4, 'row_count': 5, 'estimated_worker_count_based_on_memory_usage': 190}.
2025-05-09 15:28:15 -0700   24532 execution.bulk     INFO     Process name(SpawnProcess-4)-Process id(24524)-Line number(0) start execution.
2025-05-09 

[2025-05-09 15:28:25 -0700][promptflow._core.entry_meta_generator][WARNING] - Generate meta in current process and timeout won't take effect. Please handle timeout manually outside current process.
[2025-05-09 15:28:25 -0700][promptflow._core.entry_meta_generator][WARNING] - Generate meta in current process and timeout won't take effect. Please handle timeout manually outside current process.
[2025-05-09 15:28:25 -0700][promptflow._core.entry_meta_generator][WARNING] - Generate meta in current process and timeout won't take effect. Please handle timeout manually outside current process.
[2025-05-09 15:28:25 -0700][promptflow._core.entry_meta_generator][WARNING] - Generate meta in current process and timeout won't take effect. Please handle timeout manually outside current process.
[2025-05-09 15:28:25 -0700][promptflow._core.entry_meta_generator][WARNING] - Generate meta in current process and timeout won't take effect. Please handle timeout manually outside current process.
[2025-05-0

2025-05-09 15:28:25 -0700   24532 execution.bulk     INFO     Current thread is not main thread, skip signal handler registration in BatchEngine.
2025-05-09 15:28:25 -0700   24532 execution.bulk     INFO     Finished 5 / 5 lines.
2025-05-09 15:28:25 -0700   24532 execution.bulk     INFO     Average execution time for completed lines: 0.05 seconds. Estimated time for incomplete lines: 0.0 seconds.
======= Run Summary =======

Run name: "azure_ai_evaluation_evaluators_blue_score_20250509_152825_268461"
Run status: "Completed"
Start time: "2025-05-09 15:28:25.297951-07:00"
Duration: "0:00:01.480702"
Output path: "C:\Users\anksing\.promptflow\.runs\azure_ai_evaluation_evaluators_blue_score_20250509_152825_268461"

2025-05-09 15:28:25 -0700   24532 execution.bulk     INFO     Current thread is not main thread, skip signal handler registration in BatchEngine.
2025-05-09 15:28:25 -0700   24532 execution.bulk     INFO     Finished 5 / 5 lines.
2025-05-09 15:28:25 -0700   24532 execution.bulk  

'AI Foundry Studio URL: https://ai.azure.com/build/evaluation/57c73116-38cf-43a7-b366-c7f265102c81?wsid=/subscriptions/fac34303-435d-4486-8c3f-7094d82a0b60/resourceGroups/rg-anksingai/providers/Microsoft.MachineLearningServices/workspaces/anksing-eval-agents'

2025-05-09 15:30:01 -0700   24532 execution.bulk     INFO     Current thread is not main thread, skip signal handler registration in BatchEngine.
2025-05-09 15:30:01 -0700   24532 execution          WARNING  Starting run without column mapping may lead to unexpected results. Please consult the following documentation for more information: https://aka.ms/pf/column-mapping
2025-05-09 15:30:01 -0700   24532 execution.bulk     INFO     Current system's available memory is 43749.0859375MB, memory consumption of current process is 251.98828125MB, estimated available worker count is 43749.0859375/251.98828125 = 173
2025-05-09 15:30:01 -0700   24532 execution.bulk     INFO     Set process count to 4 by taking the minimum value among the factors of {'default_worker_count': 4, 'row_count': 5, 'estimated_worker_count_based_on_memory_usage': 173}.
2025-05-09 15:30:07 -0700   24532 execution.bulk     INFO     Process name(SpawnProcess-11)-Process id(21716)-Line number(0) start execution.
2025-05-09

### AI Foundry URL to view results

In [ ]:
f"AI Foundry Studio URL: {gpt_35_turbo_results['studio_url']}"

### Evaluate GPT 4o Mini

In [ ]:
gpt_4o_mini_results = evaluate_model("gpt-4o-mini")
pd.DataFrame(gpt_4o_mini_results["rows"])

[2025-05-09 15:30:01 -0700][promptflow._sdk._orchestrator.run_submitter][INFO] - Submitting run azure_ai_evaluation_evaluators_TARGET_20250509_152954_712426, log path: C:\Users\anksing\.promptflow\.runs\azure_ai_evaluation_evaluators_TARGET_20250509_152954_712426\logs.txt
[2025-05-09 15:30:20 -0700][promptflow._core.entry_meta_generator][WARNING] - Generate meta in current process and timeout won't take effect. Please handle timeout manually outside current process.
[2025-05-09 15:30:20 -0700][promptflow._core.entry_meta_generator][WARNING] - Generate meta in current process and timeout won't take effect. Please handle timeout manually outside current process.
[2025-05-09 15:30:20 -0700][promptflow._core.entry_meta_generator][WARNING] - Generate meta in current process and timeout won't take effect. Please handle timeout manually outside current process.
[2025-05-09 15:30:20 -0700][promptflow._core.entry_meta_generator][WARNING] - Generate meta in current process and timeout won't take

2025-05-09 15:30:31 -0700   24532 execution.bulk     INFO     Finished 1 / 5 lines.
2025-05-09 15:30:31 -0700   24532 execution.bulk     INFO     Average execution time for completed lines: 9.91 seconds. Estimated time for incomplete lines: 39.64 seconds.
2025-05-09 15:30:31 -0700   24532 execution.bulk     INFO     Finished 2 / 5 lines.
2025-05-09 15:30:31 -0700   24532 execution.bulk     INFO     Average execution time for completed lines: 5.02 seconds. Estimated time for incomplete lines: 15.06 seconds.
2025-05-09 15:30:31 -0700   24532 execution.bulk     INFO     Finished 1 / 5 lines.
2025-05-09 15:30:31 -0700   24532 execution.bulk     INFO     Average execution time for completed lines: 10.44 seconds. Estimated time for incomplete lines: 41.76 seconds.
2025-05-09 15:30:32 -0700   24532 execution.bulk     INFO     Finished 3 / 5 lines.
2025-05-09 15:30:32 -0700   24532 execution.bulk     INFO     Average execution time for completed lines: 3.63 seconds. Estimated time for incomple